<a href="https://colab.research.google.com/github/pccalegari/exemplos-CN/blob/main/unidade4_cn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>





# Unidade 4 - Aproximação de funções




Nesta unidade, vamos estudar métodos para aproximar funções e dados tabelados. Veremos duas maneiras de aproximar funções: a **Interpolação** e o **Método dos Mínimos Quadrados**. A seguir, apresentamos alguns exemplos de resolução em *Python*. Vamos utilizar as bibliotecas [numpy](https://numpy.org/) e [scipy](https://scipy.github.io/devdocs/index.html).

# Método dos Mínimos Quadrados


Objetivo: Minimizar resíduo (erro).

Para obter a melhor aproximação vamos minimizar a função:

$$  r({\bf a}) = \sum_{i=1}^m r_i({\bf a})^2,$$

sendo o resíduo $r_i({\bf a}) = f(x_i,{\bf a}) - y_i$, com $(x_i,y_i)$ dados e $m$ tamanho dos vetores $x$ e $y$.

Para ajustar os parâmetros da função $r({\bf a})$ vamos utilizar as bibliotecas *numpy* e *scipy*.


**Exemplo (1)**

Ajustar os dados abaixo usando uma reta (Regressão linear):

$$\begin{array}{c|ccccc}
x_i & 0 & 0.25 & 0.5 & 0.75 & 1\\
\hline
f(x_i) & 1 & 1.2840 & 1.6487 & 2.1170 & 2.7183 \\
\end{array}$$

*Solução:*

A função aproximadora é dada por $g(x)=a_0 + a_1x$. A tabela possui $5$ pontos portanto $m=5$.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import optimize
from scipy import interpolate
from numpy import linalg as LA

def flinear(x, a):
    y = a[0] + a[1]*x
    #y = a[0]*x + a[1]*np.cos(x)
    return y

def rlinear(a, x, y):
    return flinear(x, a) - y

xi = np.array([0, 0.25, 0.5, 0.75 , 1])
yi = np.array([1, 1.2840, 1.6487, 2.117, 2.7183])
#Exemplo 1 (aula)
#xi = np.array([0, 1, 2, 3, 4])
#yi = np.array([0, 1, 1, 4, 4])

#Exemplo 2 (aula)
#xi = np.array([0, 1.5, 3, 4.5, 6])
#yi = np.array([1, 1.57, 2, 4.3, 7])
m = len(xi)

v_1 = np.ones(m)

#Sistema para o ajuste por reta
A = np.zeros((2,2))
b = np.zeros(2)
A[0,0] = np.vdot(v_1,v_1)
A[1,0] = np.vdot(v_1,xi)
A[0,1] = A[1,0]
A[1,1] = np.vdot(xi, xi)
b[0] = np.vdot(v_1, yi)
b[1] = np.vdot(xi, yi)

#Sistema doa ajuste g(x) = a0x + a1cos(x)
#A[0,0] = np.vdot(xi, xi)
#A[1,0] = np.vdot(xi, np.cos(xi))
#A[0,1] = A[1,0]
#A[1,1] = np.vdot(np.cos(xi), np.cos(xi))
#b[0] = np.vdot(xi, yi)
#b[1] = np.vdot(np.cos(xi), yi)
print(A)
a = np.linalg.solve(A, b)
print(a)
res = 0
for i in range(m):
   res += (rlinear(a, xi[i], yi[i])**2)
print("residuo = ", res)

x_test = np.linspace(xi[0], xi[m-1], 100)

y_lin = flinear(x_test, a)

plt.figure(1)
plt.plot(xi, yi, 'o', label='Experimento')
plt.plot(x_test, y_lin, label='Ajuste linear')
plt.xlabel("x")
plt.ylabel("y")
plt.legend()
plt.show()